In [1]:
# Libraries that have been used 
import pandas as pd
import seaborn as sns
import numpy as np
import re
from sklearn.preprocessing import MinMaxScaler
from sklearn.decomposition import PCA
from datetime import datetime
import matplotlib.pyplot as plt
import itertools

from imblearn.over_sampling import SMOTE

# Change the file path based on your own configuration!!
file_path = "/home/arik/ML_Projects/Kaggle/Expresso Churn Prediction Challenge/"

In [2]:
#Instances that have churn=1, control variable of null percentage.
percentage_of_nulls_churn1 = 50

In [3]:
# Let's load the data.
train = pd.read_csv(file_path + "data/Train.csv", encoding='cp1252')
test = pd.read_csv(file_path + "data/Test.csv", encoding='cp1252')
variable_definitions = pd.read_csv(file_path + "data/VariableDefinitions.csv")

In [4]:
# Variable defitions are given in here that could be useful to consider the variables.
variable_definitions.head(train.shape[1])

,Variable Definitions,Unnamed: 1,Unnamed: 2
0,NaN,French,English
1,NaN,Le dataset churn comprend 19 variables dont 15...,The churn dataset includes 19 variables includ...
2,user_id,NaN,NaN
3,REGION,la localité de chaque client,the location of each client
4,TENURE,la durée dans le reseau,duration in the network
5,MONTANT,montant de recharge,top-up amount
6,FREQUENCE_RECH,nombre de fois que le client a fait une recharge,number of times the customer refilled
7,REVENUE,revenu mensuel de chaque client,monthly income of each client
8,ARPU_SEGMENT,revenu sur 90 jours/3,income over 90 days / 3
9,FREQUENCE,nombre de fois que client à fait un revenu,number of times the client has made an income


In [5]:
(train.isna().sum() * 100 / train.shape[0]).sort_values()

user_id            0.000000
REGULARITY         0.000000
MRG                0.000000
CHURN              0.000000
TENURE             0.000000
REVENUE           33.706213
ARPU_SEGMENT      33.706213
FREQUENCE         33.706213
FREQUENCE_RECH    35.131018
MONTANT           35.131018
ON_NET            36.520774
REGION            39.428044
ORANGE            41.561191
FREQ_TOP_PACK     41.902223
TOP_PACK          41.902223
DATA_VOLUME       49.229776
TIGO              59.887988
ZONE1             92.120835
ZONE2             93.648052
dtype: float64

In [6]:
train.drop(["MRG", "ZONE1", "ZONE2"], axis = 1, inplace = True)
test.drop(["MRG", "ZONE1", "ZONE2"], axis = 1, inplace = True)

In [7]:
train.isnull().sum(axis=1) * 100 / train.shape[1]

0           0.00
1          75.00
2          12.50
3           0.00
4          12.50
           ...  
2154043    75.00
2154044     0.00
2154045    75.00
2154046     6.25
2154047    68.75
Length: 2154048, dtype: float64

In [8]:
train["null_percentage"] = train.isnull().sum(axis = 1) * 100 / train.shape[1]

In [9]:
train[train["null_percentage"] > percentage_of_nulls_churn1]

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN,null_percentage
1,00000cb4a5d760de88fecb38e2f71b7bec52e834,NaN,I 18-21 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4,NaN,NaN,1,75.00
6,00002b0ed56e2c199ec8c3021327229afa70f063,LOUGA,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,0,68.75
10,0000527d276a6ba8b02810cc2c1d60d25e650f5f,DAKAR,K > 24 month,200.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,56.25
12,000068fe2aad763fb2667d253fa4c9e19e3b0636,NaN,G 12-15 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN,1,75.00
16,0000a0d034eb5893f9d50fc4c038fac22e5c8e80,KOLDA,K > 24 month,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,37,NaN,NaN,0,62.50
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2154039,ffffc38e1c3cb77a88941e739c358fd96bce3238,DAKAR,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,25.0,NaN,6,NaN,NaN,0,62.50
2154041,ffffd1d48dd02c059c82c70b8793c8dfa3d09593,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,0,75.00
2154043,ffffe85215ddc71a84f95af0afb0deeea90e6967,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6,NaN,NaN,0,75.00
2154045,fffff172fda1b4bb38a95385951908bb92379809,NaN,K > 24 month,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1,NaN,NaN,1,75.00


In [10]:
train[train["null_percentage"] > percentage_of_nulls_churn1].CHURN.value_counts()

0    399842
1    335600
Name: CHURN, dtype: int64

In [11]:
drop_index = train[(train["null_percentage"] > percentage_of_nulls_churn1) & (train["CHURN"] == 0)].index

In [12]:
train = train.drop(list(drop_index)).reset_index(drop = True)
train.drop("null_percentage", axis = 1, inplace = True)

In [13]:
# Unique Row-wise null percentages as index and frequencies as values for CHURN=1 
(train[train.CHURN == 1].isnull().sum(axis=1) * 100 / train[train.CHURN == 1].shape[1]).value_counts()

75.00    262233
68.75     37899
62.50     23211
12.50     16267
18.75     13791
6.25      12731
56.25     12257
25.00      9523
31.25      5207
37.50      3572
43.75      3392
50.00      2733
0.00       1170
dtype: int64

In [14]:
nullpercentage_churn_1 = pd.Series(train[train.CHURN == 1].isnull().sum(axis=1) * 100 / train[train.CHURN == 1].shape[1])

In [15]:
churn1_subset = train.iloc[nullpercentage_churn_1[nullpercentage_churn_1 < percentage_of_nulls_churn1].index]

In [16]:
nullpercentage_churn_1.shape[0]

403986

In [17]:
churn1_to_be_dropped = pd.Series(list(set(list(nullpercentage_churn_1.index)) - set(list(churn1_subset.index)))).sort_values()

In [18]:
train.drop(index=churn1_to_be_dropped, inplace=True)

In [19]:
train

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,54,On net 200F=Unlimited _call24H,8.0,0
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,11,Mixt 250F=Unlimited_call24H,2.0,0
5,0000296564272665ccd2925d377e124f3306b01e,LOUGA,K > 24 month,8500.0,17.0,9000.0,3000.0,18.0,NaN,252.0,70.0,91.0,62,MIXT:500F= 2500F on net _2500F off net;2d,18.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754199,ffffb2b8b63959b8a374e2a2ccaf2b9e521879ad,NaN,K > 24 month,1000.0,2.0,1000.0,333.0,2.0,0.0,2.0,12.0,3.0,12,All-net 500F=2000F;5d,2.0,0
1754200,ffffccdae4d9097c20f95e87f5c89845cab4eff3,SAINT-LOUIS,K > 24 month,2000.0,4.0,1997.0,666.0,5.0,0.0,57.0,1.0,NaN,21,All-net 500F=2000F;5d,2.0,0
1754201,ffffd3057e31ff19496a3c00397a9a67d5037c52,DAKAR,K > 24 month,4800.0,4.0,4800.0,1600.0,14.0,7400.0,2.0,12.0,NaN,62,"Data:1000F=2GB,30d",3.0,0
1754202,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,55,"Data: 200 F=100MB,24H",9.0,0


In [20]:
train[train.CHURN == 1]

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
35,00013be0551465ba9cd686f10aa2f0a4f67f4fe6,NaN,K > 24 month,6300.0,12.0,6299.0,2100.0,11.0,NaN,47.0,116.0,5.0,32,All-net 500F=2000F;5d,10.0,1
50,0001f8184f532f1ade411e1f1c285d60124bacda,NaN,K > 24 month,3000.0,4.0,3002.0,1001.0,8.0,266.0,1.0,8.0,NaN,32,"Data:1000F=2GB,30d",2.0,1
52,00020666991bbb333a85a5b1b5bf7e220d7ea788,NaN,K > 24 month,1500.0,3.0,1980.0,660.0,5.0,4341.0,1.0,0.0,NaN,29,"Data:490F=1GB,7d",4.0,1
67,00027e73f9c315768a1c9fc3ecb600d0bb5ecd86,NaN,K > 24 month,9850.0,32.0,9639.0,3213.0,38.0,2700.0,4.0,10.0,8.0,49,"Data: 200 F=100MB,24H",17.0,1
72,0002ada1719bc6939275e1e66f9a15a1ea28a31a,KAOLACK,I 18-21 month,500.0,1.0,500.0,167.0,3.0,NaN,3.0,1.0,NaN,7,NaN,NaN,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754005,fff79d914cd3ff134807904366177baa83a85fb6,NaN,K > 24 month,2800.0,4.0,2299.0,766.0,6.0,2730.0,3.0,52.0,3.0,21,"Data:1000F=2GB,30d",1.0,1
1754027,fff887ff651f50cfc66b7cc956283f31e942e78a,NaN,K > 24 month,4800.0,27.0,4809.0,1603.0,28.0,3540.0,1.0,10.0,NaN,45,"Data: 100 F=40MB,24H",21.0,1
1754041,fff90edf06d2ab42a3616b1dfc7c91f6a1eabb80,NaN,K > 24 month,5000.0,4.0,7000.0,2333.0,4.0,12078.0,0.0,NaN,NaN,43,"Data:1000F=2GB,30d",1.0,1
1754092,fffb4aa50671c63f82628aa233fe6fc97fbbefd0,NaN,K > 24 month,1000.0,2.0,1000.0,333.0,3.0,1691.0,NaN,NaN,NaN,3,"Data:490F=1GB,7d",2.0,1


In [21]:
train[train.CHURN == 0]

,user_id,REGION,TENURE,MONTANT,FREQUENCE_RECH,REVENUE,ARPU_SEGMENT,FREQUENCE,DATA_VOLUME,ON_NET,ORANGE,TIGO,REGULARITY,TOP_PACK,FREQ_TOP_PACK,CHURN
0,00000bfd7d50f01092811bc0c8d7b0d6fe7c3596,FATICK,K > 24 month,4250.0,15.0,4251.0,1417.0,17.0,4.0,388.0,46.0,1.0,54,On net 200F=Unlimited _call24H,8.0,0
2,00001654a9d9f96303d9969d0a4a851714a4bb57,NaN,K > 24 month,3600.0,2.0,1020.0,340.0,2.0,NaN,90.0,46.0,7.0,17,On-net 1000F=10MilF;10d,1.0,0
3,00001dd6fa45f7ba044bd5d84937be464ce78ac2,DAKAR,K > 24 month,13500.0,15.0,13502.0,4501.0,18.0,43804.0,41.0,102.0,2.0,62,"Data:1000F=5GB,7d",11.0,0
4,000028d9e13a595abe061f9b58f3d76ab907850f,DAKAR,K > 24 month,1000.0,1.0,985.0,328.0,1.0,NaN,39.0,24.0,NaN,11,Mixt 250F=Unlimited_call24H,2.0,0
5,0000296564272665ccd2925d377e124f3306b01e,LOUGA,K > 24 month,8500.0,17.0,9000.0,3000.0,18.0,NaN,252.0,70.0,91.0,62,MIXT:500F= 2500F on net _2500F off net;2d,18.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1754199,ffffb2b8b63959b8a374e2a2ccaf2b9e521879ad,NaN,K > 24 month,1000.0,2.0,1000.0,333.0,2.0,0.0,2.0,12.0,3.0,12,All-net 500F=2000F;5d,2.0,0
1754200,ffffccdae4d9097c20f95e87f5c89845cab4eff3,SAINT-LOUIS,K > 24 month,2000.0,4.0,1997.0,666.0,5.0,0.0,57.0,1.0,NaN,21,All-net 500F=2000F;5d,2.0,0
1754201,ffffd3057e31ff19496a3c00397a9a67d5037c52,DAKAR,K > 24 month,4800.0,4.0,4800.0,1600.0,14.0,7400.0,2.0,12.0,NaN,62,"Data:1000F=2GB,30d",3.0,0
1754202,ffffeaaa9289cdba0ac000f0ab4b48f4aa74ed15,THIES,K > 24 month,6100.0,15.0,5800.0,1933.0,15.0,621.0,26.0,40.0,40.0,55,"Data: 200 F=100MB,24H",9.0,0


In [22]:
train.to_csv(file_path + "data/train_prunned.csv", index = False, sep=",")
test.to_csv(file_path + "data/test_prunned.csv", index = False, sep=",")